Test Cases for Fluxonium circuit :
* anHarmonicity flatness across flux spectrum
* analytically related to Ej

In [1]:
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import lossAnharmonicity,lossTransitionFlatness
from torch import tensor
from numpy import arange,linspace
from DiSuQ.utils import plotCompare
from torch import set_num_threads
set_num_threads(32)

* initial circuit complies with oscillator basis
* spectrum is invariant with higher truncation

In [2]:
basis = {'O':[512],'I':[],'J':[]}; rep = 'K'
#basis = [512]; rep = 'Q'
circuit = models.fluxonium(basis,El=10.,Ec=100.,Ej=50,sparse=False)
print(circuit.circuitComponents())

{'Cap': 99.99999237060547, 'JJ': 50.000003814697266, 'I': 9.999999046325684}


In [3]:
flux_range = tensor(linspace(0,1,32,endpoint=True))

#### pre-optimization Fluxonium

In [4]:
flux_profile = [[flux] for flux in flux_range]
flux_point = ('I')

In [5]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson

In [6]:
#H_LC = circuit.chargeHamiltonianLC()
#H_J = circuit.josephsonCharge

In [7]:
pre_E0,pre_Ex = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=[1,2])

In [8]:
pre_Ex = pre_Ex.detach().numpy()

In [29]:
plotCompare(flux_range,{'Ist':pre_Ex[0],'IInd':pre_Ex[1],'anharmonicity':pre_Ex[1]-2*pre_Ex[0]},
            'Excitation Profile : pre-optimization','external flux','energy(GHz)',export='pdf',size=(600,800))

#### optimization

In [12]:
flux_profile = [{'I':flux} for flux in flux_range]

In [13]:
optim = OrderingOptimization(circuit,representation=rep)
dLogs,dParams,dCircuit = optim.optimization(lossTransitionFlatness,flux_profile,iterations=50,lr=.0005)

In [30]:
plotCompare(dLogs.index,dLogs[['loss']],'Optimization-Flux Sensitivity','iteration',export='pdf',size=(600,800))

In [31]:
plotCompare(dCircuit.index,dCircuit,'Optimization-Circuit Parameters',"iteration","energy(GHz)",export='pdf',size=(600,800))

Analytically the solution is $JJ=0$ \
However, the optimization stagnates asymptotically \
and therefore gradient descent algorithm improper

#### post-optimization Fluxonium

In [34]:
dCircuit.iloc[0]

Cap     0.004813
JJ     25.371658
I       0.002710
Name: 0, dtype: float64

In [35]:
dCircuit.iloc[-1]

Cap    0.004384
JJ     5.575057
I      0.003024
Name: 49, dtype: float64

In [21]:
flux_profile = [[flux] for flux in flux_range]
flux_point = ('I')

In [22]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson

In [23]:
post_E0,post_Ex = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=[1,2])

In [24]:
post_Ex = post_Ex.detach().numpy()

In [32]:
plotCompare(flux_range,{'Ist':post_Ex[0],'IInd':post_Ex[1],'anharmonicity':post_Ex[1]-2*post_Ex[0]},
            'Excitation Profile : post-optimization','external flux','energy(GHz)',export='pdf',size=(600,800))

In [26]:
print(circuit.circuitComponents())

{'Cap': 114.05835723876953, 'JJ': 5.575057029724121, 'I': 8.377640724182129}


## Circuit Comparison

In [33]:
plotCompare(flux_range,{'post':post_Ex[1]-2*post_Ex[0],'pre':pre_Ex[1]-2*pre_Ex[0]},
            'Flux Profile-Anharmonicity','external flux','anharmonicity(GHz)',export='pdf',size=(600,800))